# Lấy dữ liệu vĩ mô

In [2]:
# Install libraries if needed
!pip install requests pandas --quiet

import requests
import pandas as pd
from functools import reduce

# ✅ Country - Only Vietnam
country = "VNM"

# ✅ Indicators required (Updated with IIP)
indicators = {
    "GDP Growth (%)": "NY.GDP.MKTP.KD.ZG",
    "CPI (Inflation, annual %)": "FP.CPI.TOTL.ZG",
    "Exchange Rate (USD/VND)": "PA.NUS.FCRF",
    "Interest Rate (%) (Lending Rate Proxy)": "FR.INR.LEND",
    "FDI Inflow (US$)": "BX.KLT.DINV.CD.WD",
    "Industrial Production Growth (IIP %)": "IP.PRD.INDX.ZG"
}

start_year = 2010
end_year = 2024

# ✅ World Bank API data fetch function
def get_wb_data(country_code, indicator_code, start, end):
    url = (
        f"https://api.worldbank.org/v2/country/{country_code}/indicator/"
        f"{indicator_code}?format=json&date={start}:{end}&per_page=2000"
    )
    r = requests.get(url)
    data = r.json()

    # Nếu không có dữ liệu
    if len(data) < 2 or not isinstance(data[1], list) or len(data[1]) == 0:
        print(f"⚠️ Warning: No data for indicator {indicator_code}")
        return pd.DataFrame()

    df = pd.DataFrame(data[1])[["date", "value"]]
    df["Year"] = df["date"].astype(int)
    df.drop(columns=["date"], inplace=True)
    df.rename(columns={"value": indicator_code}, inplace=True)
    return df

# ✅ Retrieve all indicators for Vietnam
data_frames = []
for name, code in indicators.items():
    print(f"Fetching {name} ...")
    df_temp = get_wb_data(country, code, start_year, end_year)
    if df_temp.empty:
        continue  # bỏ DataFrame rỗng
    df_temp.rename(columns={code: name}, inplace=True)
    data_frames.append(df_temp)

# ✅ Merge into a single table, chỉ khi có DataFrame tồn tại
if data_frames:
    df = reduce(lambda left, right: pd.merge(left, right, on="Year", how="outer"), data_frames)
    df = df.sort_values("Year").reset_index(drop=True)
else:
    df = pd.DataFrame()
    print("❌ No data available for any indicator!")

# ✅ Formatting display
pd.options.display.float_format = '{:,.2f}'.format

# ✅ Export Excel nếu có dữ liệu
if not df.empty:
    output_filename = "Vietnam_Macro_Indicators_2010_2024.xlsx"
    df.to_excel(output_filename, index=False)
    print("✅ File exported successfully:", output_filename)
else:
    print("❌ No Excel file exported, data empty.")

df


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Fetching GDP Growth (%) ...
Fetching CPI (Inflation, annual %) ...
Fetching Exchange Rate (USD/VND) ...
Fetching Interest Rate (%) (Lending Rate Proxy) ...
Fetching FDI Inflow (US$) ...
Fetching Industrial Production Growth (IIP %) ...
⚠️ Warning: No data for indicator IP.PRD.INDX.ZG


,GDP Growth (%),Year,"CPI (Inflation, annual %)",Exchange Rate (USD/VND),Interest Rate (%) (Lending Rate Proxy),FDI Inflow (US$)
0,6.42,2010,9.21,"18,612.92",13.14,8000000000
1,6.41,2011,18.68,"20,509.75",16.95,7430000000
2,5.50,2012,9.09,"20,828.00",13.47,8368000000
3,5.55,2013,6.59,"20,933.42",10.37,8900000000
4,6.42,2014,4.08,"21,148.00",8.66,9200000000
5,6.99,2015,0.63,"21,697.57",7.12,11800000000
6,6.69,2016,2.67,"21,935.00",6.96,12600000000
7,6.94,2017,3.52,"22,370.09",7.07,14100000000
8,7.47,2018,3.54,"22,602.05",7.37,15500000000
9,7.36,2019,2.80,"23,050.24",7.71,16120000000


In [3]:
# Lấy dữ liệu giá lịch sử

In [4]:
# thư viện vnstock
!pip install git+https://github.com/thinh-vu/vnstock.git@main

  Cloning https://github.com/thinh-vu/vnstock.git (to revision main) to /tmp/pip-req-build-_t5zp9wy
  Running command git clone --filter=blob:none --quiet https://github.com/thinh-vu/vnstock.git /tmp/pip-req-build-_t5zp9wy
  Resolved https://github.com/thinh-vu/vnstock.git to commit 7bc8ec5eeda5c107995975d985306405ccbc512e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.2/539.2 kB 20.3 MB/s eta 0:00:00
  Created wheel for vnstock: filename=vnstock-3.2.7-py3-none-any.whl size=210776 sha256=b8f2f8b825433037fe0e96653786e28a71251ca23926f99a47cc618ef907f157
  Stored in directory: /tmp/pip-ephem-wheel-cache-3o_jxq8l/wheels/bf/e3/28/3d02eb0adfc4664c0a84b3900fc986b516df7015f08a33000b
Successfully built vnstock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [vnstock]3/14 [vnstock]]



In [5]:
!pip install vnstock


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
!pip install vnstock3 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [vnstock3]2/3 [vnstock3]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
# import toàn bộ ừ vnstock
from vnstock import *

Non-200 response fetching content: 502



## 👋 Chào mừng bạn đến với Vnstock!
Cảm ơn bạn đã sử dụng package phân tích chứng khoán #1 tại Việt Nam
* Tài liệu: [Sổ tay hướng dẫn](https://vnstocks.com/docs)
* Cộng đồng: [Nhóm Facebook](https://facebook.com/groups/vnstock.official)
Khám phá các tính năng mới nhất và tham gia cộng đồng để nhận hỗ trợ.
                

In [8]:
from vnstock import Listing
listing = Listing()
listing.all_symbols()

,symbol,organ_name
0,YTC,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...
1,YEG,Công ty Cổ phần Tập đoàn Yeah1
2,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
3,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
4,XPH,Công ty Cổ phần Xà phòng Hà Nội
...,...,...
1722,AAS,Công ty Cổ phần Chứng khoán SmartInvest
1723,AAM,Công ty Cổ Phần Thủy Sản MeKong
1724,AAH,Công ty Cổ phần Hợp Nhất
1725,AAA,Công ty Cổ phần Nhựa An Phát Xanh


In [9]:
# dữ liệu giá lịch sử của HPG, NKG, HSG, POM, TVN

In [10]:
hpg = Vnstock().stock(symbol='HPG', source='VCI')
hpg.quote.history(start='2022-01-01', end='2025-10-8', interval='1D')

,time,open,high,low,close,volume
0,2021-10-28,33.20,33.37,32.85,33.25,21923800
1,2021-10-29,33.25,33.31,32.74,32.74,23878000
2,2021-11-01,32.74,32.74,31.93,31.93,45218500
3,2021-11-02,31.82,32.34,31.65,31.82,25669500
4,2021-11-03,32.22,32.28,31.70,31.88,29867100
...,...,...,...,...,...,...
980,2025-10-03,28.10,28.10,27.35,27.65,58123500
981,2025-10-06,28.10,29.20,28.10,29.20,71269500
982,2025-10-07,29.20,29.35,29.00,29.00,41398600
983,2025-10-08,29.25,29.40,28.75,29.20,59953000


In [11]:
nkg = Vnstock().stock(symbol='NKG', source='VCI')
nkg.quote.history(start='2022-01-01', end='2025-10-8', interval='1D')

,time,open,high,low,close,volume
0,2021-10-28,34.46,35.40,34.15,34.84,6005100
1,2021-10-29,34.46,34.78,33.52,34.09,19314300
2,2021-11-01,33.84,33.96,33.02,33.40,13646000
3,2021-11-02,33.09,34.28,32.96,33.21,10912800
4,2021-11-03,33.21,33.59,31.64,31.64,18305400
...,...,...,...,...,...,...
980,2025-10-03,17.05,17.10,16.60,16.70,12484100
981,2025-10-06,16.90,17.80,16.85,17.65,7403200
982,2025-10-07,17.80,17.80,17.30,17.30,7089200
983,2025-10-08,17.60,17.75,17.20,17.70,10543900


In [12]:
hsg = Vnstock().stock(symbol='HSG', source='VCI')
hsg.quote.history(start='2022-01-01', end='2025-10-8', interval='1D')

,time,open,high,low,close,volume
0,2021-10-28,37.23,37.23,36.57,36.77,6865500
1,2021-10-29,37.00,37.07,35.88,36.31,11471500
2,2021-11-01,36.38,36.46,35.12,35.23,17399100
3,2021-11-02,35.23,35.96,34.97,35.08,11151200
4,2021-11-03,35.23,35.39,34.08,34.43,15160400
...,...,...,...,...,...,...
980,2025-10-03,18.60,18.60,18.05,18.05,5984600
981,2025-10-06,18.30,19.00,18.15,18.80,4100400
982,2025-10-07,18.95,18.95,18.50,18.50,2598400
983,2025-10-08,18.90,18.95,18.50,18.80,3554400


In [13]:
pom = Vnstock().stock(symbol='POM', source='VCI')
pom.quote.history(start='2022-01-01', end='2025-10-8', interval='1D')

,time,open,high,low,close,volume
0,2020-10-29,4.85,5.50,4.85,5.25,29050
1,2020-10-30,5.25,5.50,5.16,5.35,1820
2,2020-11-02,5.35,5.50,5.35,5.40,50000
3,2020-11-03,5.40,5.50,5.30,5.40,26640
4,2020-11-04,5.06,5.50,5.06,5.40,21290
...,...,...,...,...,...,...
980,2025-09-05,2.00,2.30,2.00,2.30,1651400
981,2025-09-12,2.10,2.40,2.10,2.30,574700
982,2025-09-19,2.20,2.30,1.90,2.00,2273100
983,2025-09-26,2.00,2.00,1.80,1.90,1237600


In [14]:
tvn = Vnstock().stock(symbol='TVN', source='VCI')
tvn.quote.history(start='2022-01-01', end='2025-10-8', interval='1D')

,time,open,high,low,close,volume
0,2021-10-28,18.24,18.81,17.86,18.62,2758996
1,2021-10-29,18.62,18.81,18.24,18.43,2193099
2,2021-11-01,18.52,18.62,17.95,18.24,2057915
3,2021-11-02,18.24,18.52,17.86,18.24,1765832
4,2021-11-03,18.24,18.43,17.67,17.86,1890574
...,...,...,...,...,...,...
980,2025-10-03,8.10,8.10,7.90,8.00,790400
981,2025-10-06,8.00,8.40,8.00,8.30,272200
982,2025-10-07,8.30,8.30,8.00,8.10,212800
983,2025-10-08,8.20,8.30,8.10,8.20,246100


In [15]:
# dữ liệu báo cáo tài chính và chỉ số tài chính

In [16]:
# truy xuất thông tin công ty
import pandas as pd
from vnstock import Vnstock
import time

# 1. Danh sách 5 mã thép của bạn
tickers_list = ['HPG', 'NKG', 'HSG', 'POM', 'TVN']

# 2. Tạo một danh sách (list) rỗng để lưu kết quả
all_overviews = []

print(f"Bắt đầu tải thông tin tổng quan cho {len(tickers_list)} công ty...")

# 3. Lặp qua từng mã trong danh sách
for ticker in tickers_list:
    print(f"Đang tải: {ticker}...")
    try:
        # Khởi tạo đối tượng 'company' cho từng mã
        company_helper = Vnstock().stock(symbol=ticker, source='VCI').company
        
        # Gọi .overview() để lấy DataFrame (chỉ có 1 hàng)
        overview_df = company_helper.overview()
        
        # Thêm DataFrame 1 hàng này vào danh sách
        all_overviews.append(overview_df)
        
    except Exception as e:
        # Báo lỗi nếu mã đó không tải được (ví dụ: VCI không có dữ liệu)
        print(f"  -> Lỗi khi tải {ticker}: {e}")
        
    # Tạm dừng một chút để không gọi API quá nhanh
    time.sleep(0.3)

# 4. Gộp tất cả các DataFrame nhỏ trong danh sách thành 1 DataFrame lớn
if all_overviews:
    final_df = pd.concat(all_overviews, ignore_index=True)
    
    print("\n--- HOÀN TẤT TẢI DỮ LIỆU ---")
    # In ra DataFrame cuối cùng
    print(final_df)
else:
    print("Không tải được dữ liệu cho bất kỳ công ty nào.")

Bắt đầu tải thông tin tổng quan cho 5 công ty...
Đang tải: HPG...


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


Đang tải: NKG...


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


Đang tải: HSG...


...

Đang tải: POM...


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


Đang tải: TVN...


DataFrame.applymap has been deprecated. Use DataFrame.map instead.



--- HOÀN TẤT TẢI DỮ LIỆU ---
  symbol     id  issue_share  \
0    HPG  70880   7675465855   
1    NKG  72577    447570881   
2    HSG  70975    620982309   
3    POM  73254    279676336   
4    TVN  76565    678000000   

                                             history  \
0   - Năm 2000: Công ty TNHH Sắt thép Hòa Phát đư...   
1   - Ngày 23/12/2002: Công ty Cổ phần Thép Nam K...   
2   - Ngày 08/08/2001: Tiền thân là Công ty Cổ ph...   
3   - Năm 1999: Công ty Cổ phần Thép Pomina, tiền...   
4   - 1995: Tổng Công ty Thép Việt Nam được thành...   

                                     company_profile icb_name3  \
0  Công ty Cổ phần Tập đoàn Hoà Phát (HPG) là một...  Kim loại   
1  Công ty Cổ phần Thép Nam Kim (NKG) được thành ...  Kim loại   
2  Công ty Cổ phần Tập đoàn Hoa Sen (HSG), có tiề...  Kim loại   
3  Công ty Cổ phần Thép Pomina (POM) có tiền thân...  Kim loại   
4  Tổng công ty Thép Việt Nam (TVN) được thành lậ...  Kim loại   

           icb_name2              icb_name4

In [17]:
# truy xuất báo cáo tài chính
from vnstock import Vnstock
import pandas as pd

# Danh sách mã cổ phiếu cần lấy
symbols = ['HPG', 'NKG', 'HSG', 'POM', 'TVN']

# Tạo list để lưu kết quả
all_balance_sheets = []
all_income_statements = []
all_cash_flows = []
all_ratios = []

for symbol in symbols:
    print(f"Đang lấy dữ liệu cho: {symbol}")
    try:
        stock = Vnstock().stock(symbol=symbol, source='VCI')

        # Bảng cân đối kế toán - quý
        bs = stock.finance.balance_sheet(period='quarter', lang='vi', dropna=True)
        bs['symbol'] = symbol
        all_balance_sheets.append(bs)

        # KQKD - quý
        is_ = stock.finance.income_statement(period='quarter', lang='vi', dropna=True)
        is_['symbol'] = symbol
        all_income_statements.append(is_)

        # Lưu chuyển tiền tệ - quý
        cf = stock.finance.cash_flow(period='quarter', dropna=True)
        cf['symbol'] = symbol
        all_cash_flows.append(cf)

        # Chỉ số tài chính - quý
        ratio = stock.finance.ratio(period='quarter', lang='vi', dropna=True)
        ratio['symbol'] = symbol
        all_ratios.append(ratio)

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")

Đang lấy dữ liệu cho: HPG
Đang lấy dữ liệu cho: NKG
Đang lấy dữ liệu cho: HSG
Đang lấy dữ liệu cho: POM
Đang lấy dữ liệu cho: TVN


In [18]:
# Gộp tất cả lại thành DataFrame
df_balance = pd.concat(all_balance_sheets)
df_income = pd.concat(all_income_statements)
df_cashflow = pd.concat(all_cash_flows)
df_ratio = pd.concat(all_ratios)

In [19]:
df_balance

,CP,Năm,Kỳ,TÀI SẢN NGẮN HẠN (đồng),Tiền và tương đương tiền (đồng),Giá trị thuần đầu tư ngắn hạn (đồng),Các khoản phải thu ngắn hạn (đồng),Hàng tồn kho ròng,Tài sản lưu động khác,TÀI SẢN DÀI HẠN (đồng),...,Vay và nợ thuê tài chính dài hạn (đồng),Người mua trả tiền trước ngắn hạn (đồng),Vay và nợ thuê tài chính ngắn hạn (đồng),Lợi thế thương mại (đồng),Trả trước dài hạn (đồng),Tài sản dài hạn khác (đồng),Phải thu dài hạn khác (đồng),Phải thu dài hạn (đồng),symbol,Các quỹ khác
0,HPG,2025,3,95425988115582,9092562739415,"18,903,949,054,642.00",13727194725111,45623656431728,8078625164686,150745350172070,...,28356095137594,619205121131,68482383626253,"52,097,751,283.00",4390536576181,0.00,825562606009,849788287676,HPG,NaN
1,HPG,2025,2,97613904938535,10688024277258,"17,584,027,436,423.00",12334925649801,48853408114115,8153519460938,144610625980515,...,27312288660990,306667783670,66692028477152,"57,563,156,071.00",4338725174159,0.00,827053815447,900453913211,HPG,NaN
2,HPG,2025,1,88914195573724,7459035818129,"16,212,510,112,252.00",10856053423195,46603844415849,7782751804299,139947392214583,...,27256965869238,570631436241,61784512821075,"60,588,141,397.00",4261916994097,0.00,848015622228,920743791614,HPG,NaN
3,HPG,2024,4,86674276272995,6887646139852,"18,974,716,730,905.00",7647800286988,46091222189472,7072890925778,137815431280986,...,27080443256096,739178306553,55882686213459,"63,613,126,723.00",4269141694868,0.00,840594835822,923400123614,HPG,NaN
4,HPG,2024,3,80841803394106,8500998079280,"16,386,803,148,945.00",9140948029240,40197821463879,6615232672762,130544540534012,...,24517626484538,440576623880,54180722680952,"66,834,062,714.00",4347213561931,0.00,841682562478,1031145036427,HPG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,TVN,2014,1,6946388366425,964124450864,"401,539,444,839.00",1876183887241,3457757662778,246782920703,15346764858989,...,4343636221052,157420958724,6405412441033,0.00,728556399979,"86,798,782,701.00",129077290,1166049530,TVN,"110,269,231,539.00"
47,TVN,2013,4,7552600085741,1277608395309,"401,398,339,339.00",1886217176434,3674339678296,313036496363,15237091927659,...,4333193532776,151385302852,7921216001933,0.00,749165494680,"65,183,082,374.00",173502060,1388704884,TVN,"110,269,268,255.00"
48,TVN,2013,3,8393029113251,1225026296809,"210,077,929,693.00",2659801433238,3907407904854,390715548657,15387135727970,...,4767874056503,145933654813,7533109875605,0.00,695260569083,"231,405,265,558.00",281679345,1766634016,TVN,"64,245,268,255.00"
49,TVN,2013,2,8798090685886,1226033877551,"177,723,357,813.00",2459761975222,4052745162284,881826313016,15218185302903,...,4750359845620,115615477102,6986700636970,0.00,708956139715,"240,310,077,438.00",311687892,5866893278,TVN,"64,245,268,255.00"
